In [ ]:
import os
import sys
import glob
import numpy as np
import pandas as pd
import math
import gc
import cv2
from tqdm import tqdm
import time
from functools import lru_cache
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import Dataset,DataLoader
from torch.cuda.amp import autocast,GradScaler

In [ ]:
CFG={
    'seed':42,
     'model':'resnet50',
     'img_size':256,
     'epochs':10,
     'train_bs':100,
     'valid_bs':64,
     'lr':1e-3,
     'weight_decay':1e-6,
     'num_workers':2
}

In [ ]:
def seed_everything(seed):
  random.seed(seed)
  os.environ['PYTHONASHSEED']=str(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  torch.cuda.manual_seed(seed)
  torch.backends.cudnn.deterministic=True
  torch.backends.cudnn.benchmark=False
seed_everything(CFG['seed'])
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [ ]:
def get_labels(df):
  #split contact_id into separate columns.
  df['game_play']=df['contact_id'].str[:12]
  df['step']=df['contact_id'].str.split('_').str[-3].astype('int16')
  df['nfl_player_id_1']=df['contact_id'].str.split('_').str[-2]
  df['nfl_player_id_2']=df['contact_id'].str.split('_').str[-1]
  return df


labels=get_labels(pd.read_csv(f'{path}/sample_submission.csv'))

test_tracking=pd.read_csv(f'{path}/test_player_tracking.csv')

test_helmets=pd.read_csv(f'{path}/test_baseline_helmets.csv')

test_video_metadata=pd.read_csv(f'{path}/test_video_metadata.csv')

In [ ]:
!mkdir -p ../work/frames

for video in tqdm(test_helmets.video.unique()):
    if 'Endzone2' not in video:
        !ffmpeg -i /kaggle/input/nfl-player-contact-detection/test/{video} -q:v 2 -f image2 /kaggle/work/frames/{video}_%04d.jpg -hide_banner -loglevel error

In [1]:
def create_features(df,tr_tracking,merge_col='step',use_cols=['x_position','y_position']):
  output_cols=[]
  df_combo=(
      df.astype({'nfl_player_id_1':'str'})
      .merge(
          tr_tracking.astype({'nfl_player_id':'str'})[
              ['game_play',merge_col,'nfl_player_id'] + use_cols
          ],
          left_on=['game_play',merge_col,'nfl_player_id_1'],
          right_on=['game_play',merge_col,'nfl_player_id'],
          how='left'
      )
      .rename(columns={c:c+'_1'for c in use_cols})
      .drop('nfl_player_id',axis=1)
      .merge(
          tr_tracking.astype({'nfl_player_id':'str'})[
              ['game_play',merge_col,'nfl_player_id']+use_cols
          ],
          left_on=['game_play',merge_col,'nfl_player_id_2'],
          rigth_on=['game_play',merge_col,'nfl_player_id'],
          how='left',
      )
      .drop('nfl_player_id',axis=1)
      .rename(columns={c:c+'_2' for c in use_cols})
      .sort_values(['game_play',merge_col,'nfl_player_id_1','nfl_player_Id_2'])
      .reset_index(drop=True)
  )
  output_cols+=[c+'_1' for c in use_cols]
  output_cols+=[c+'_2' for c in use_cols]



  if ('x_position' in use_cols) & ('y_position' in use_cols):
    index=df_combo['x_position_2'].notnull()
    
    distance_arr=np.full(len(index),np.nan)
    tmp_distance_arr=np.sqrt(
        np.square(df_combo.loc[index,'x_position_1']-df_combo.loc[index,'y_position_1'])
        +np.square(df_combo.loc[index,'x_position_2']-df_combo.loc[index,'y_position_2']))
    )
    distance_arr[index]=tmp_distance_arr
    df_combo['distance']=distance_arr
    output_cols+=['distance']
  df_combo['G_flug']=(df_combo['nfl_player_id_2']=='G')
  output_cols +=['G_flug']
  return df_combo,output_cols

use_cols=[
    'x_position','y_position','speed','distance',
    'direction','orientation','acceleration','sa'
]
test,feature_cols=create_features(labels,test_tracking,use_cols=use_cols)
test

SyntaxError: ignored

In [ ]:
test_filterd=test.query('not distance>2').reset_index(drop=True)
test_filtered['frame']=(test_filtered['step']/10*59.94+5*59.94).astype('int')+1
test_filtered

In [ ]:
del test,labels,test_tracking
gc.collect()